In [1]:
# from urllib.request import urlopen, Request
import requests
from bs4 import BeautifulSoup

In [73]:
import pandas as pd

In [136]:
from transliterate import translit

In [139]:
from time import sleep

### Avito

In [2]:
# Что первично можно выцепить на сайте:
# Город, тип дома, кол-во комнат

In [138]:
base_url = 'https://www.avito.ru/'

In [213]:
def get_similar_flats(city, street, n_rooms, house_type):
    base_url = 'https://www.avito.ru/'
    city =  (translit(city, reversed=True)).lower()
    rooms = {
        1: '1-komnatnye',
        2: '2-komnatnye',
        3: '3-komnatnye',
        4: '4-komnatnye'
    }   
    house_types = {
        'Кирпичный': 'kirpichnyy_dom',
        'Панельный': 'panelnyy_dom',
        'Блочный': 'blochnyy_dom',
        'Монолитный': 'monolitnyy_dom',
        'Деревянный': 'derevyannyy_dom'
    }   
    search = base_url + city + '/kvartiry/prodam/' + rooms[n_rooms] + '/' + house_types[house_type] + '?q=' + '+'.join(street.split()) + '&s=104'
    print(search)
    html_doc =  requests.get(search)
    # Проверка на корректность запроса
    if html_doc.status_code != 200:
        print(html_doc.text)
          #time.sleep(int(response.headers["Retry-After"]))
    # Первая страница поиска Авито
    soup = BeautifulSoup(html_doc.text, "html.parser")
    # Нахожу все подходящие (не рекомендации) квартиры на этой странице
    flats = soup.find('div', class_ = 'items-items-kAJAg')  # получаю список квартир вместе с рекламой и рекомендациями
    flats_act = flats.find_all(attrs={"data-marker": "item"})  # баблы страничек как массив

    # Получаю сайт с каждой квартирой отдельно
    flats_sites = []
    for flat in flats_act:
        flats_sites.append('https://www.avito.ru' + flat.find(attrs={"class":"iva-item-sliderLink-bJ9Pv"})['href'])

    flat_one_info = {
                    'floor':None,
                    'all_floors':None,
                    'rooms':None,
                    'all_square':None,
                    'live_square':None,
                    'kitch_square':None,
                    'tall':None,
                    'repair':None,
                    'bathroom':None,
                    'balcony':None,
                    'window':None,
                    'dop_info':None,
                    'house_type':None,
                    'year_built':None,
                    'how_pas_lift':None,
                    'how_grz_lift':None,
                    'in_house':None,
                    'backyard':None,
                    'park':None
                    }
    df = pd.DataFrame(flat_one_info, index = [0])

    for flat in flats_sites:
        flat_i = BeautifulSoup(requests.get(flat).text, "html.parser")
        sleep(2.5)
        print(flat_i)
        flat_one_info = {
                        'floor':None,
                        'all_floors':None,
                        'rooms':None,
                        'all_square':None,
                        'live_square':None,
                        'kitch_square':None,
                        'tall':None,
                        'repair':None,
                        'bathroom':None,
                        'balcony':None,
                        'window':None,
                        'dop_info':None,
                        'house_type':None,
                        'year_built':None,
                        'how_pas_lift':None,
                        'how_grz_lift':None,
                        'in_house':None,
                        'backyard':None,
                        'park':None,
                        'price':None
                        }

        for text in flat_i.find_all('li', class_ = 'item-params-list-item'):
            label = text.text[:text.text.find(':')].split()[0]
            value = text.text[text.text.find(':') + 1:].split()
            if label == 'Этаж':
                flat_one_info['floor'] = value[0]
                flat_one_info['all_floors'] = value[2]
            elif label == 'Количество':
                flat_one_info['rooms'] = value[0]
            elif label == 'Общая':
                flat_one_info['all_square'] = value[0]
            elif label == 'Площадь':
                flat_one_info['live_square'] = value[0]
            elif label == 'Жилая':
                flat_one_info['kitchen_square'] = value[0]
            elif label == 'Высота':
                flat_one_info['tall'] = value[0]   
            elif label == 'Ремонт':
                flat_one_info['repair'] = value[0]
            elif label == 'Санузел':
                flat_one_info['bathroom'] = value[0]
            elif label == 'Балкон':
                flat_one_info['balcony'] = value[0]
            elif label == 'Вид':
                flat_one_info['window'] = ' '.join(value)
            elif label == 'Дополнительно':
                flat_one_info['dop_info'] = ' '.join(value)
            elif label == 'Тип':
                flat_one_info['house_type'] = value[0]
            elif label == 'Год':
                flat_one_info['year_built'] = value[0]
            elif label == 'Пассажирский':
                flat_one_info['how_pas_lift'] = value[0]
            elif label == 'Грузовой':
                flat_one_info['how_grz_lift'] = value[0]
            elif label == 'В':
                flat_one_info['in_house'] = ' '.join(value)
            elif label == 'Двор':
                flat_one_info['backyard'] = ' '.join(value)
            elif label == 'Парковка':
                flat_one_info['park'] = ' '.join(value) 
            flat_one_info['price'] = int(flat_i.find('span', class_ = 'js-item-price')['content'])
        df = df.append(flat_one_info, ignore_index=True)

    df.drop(index = 0,inplace=True)
    return df
# else:
#     print('Ошибка при поключении к сайту!')
#     print(html_doc)
#    return 0

In [214]:
res = get_similar_flats('Москва', 'Салтыковская', 2, 'Панельный')

https://www.avito.ru/moskva/kvartiry/prodam/2-komnatnye/panelnyy_dom?q=Салтыковская&s=104
<!DOCTYPE html> <html> <head>
    <script nonce="V9F5/8cPJmnOHKoL0nT+tQ==">
 window.dataLayer = [{"userAuth":0,"pageType":"Other"}];
 (function(w, d, s, l, i) {
 w[l] = w[l] || [];
 w[l].push({
 'gtm.start': new Date().getTime(),
 event: 'gtm.js'
 });
 var f = d.getElementsByTagName(s)[0],
 j = d.createElement(s),
 dl = l != 'dataLayer' ? '&l=' + l : '';
 j.async = true;
 j.src = '//www.googletagmanager.com/gtm.js?id=' + i + dl;
 var n=d.querySelector('[nonce]');
 n&&j.setAttribute('nonce',n.nonce||n.getAttribute('nonce'));
 f.parentNode.insertBefore(j, f);
 })(window, document, 'script', 'dataLayer', 'GTM-KP9Q9H');
 </script>
  
 <meta charset="UTF-8"> <title>Доступ временно заблокирован</title>
   <link rel="stylesheet" href="https://static.avito.ru/s/cc/styles/d1b637144138574acbdc.css">
<script nonce="V9F5/8cPJmnOHKoL0nT+tQ==" src="https://static.avito.ru/s/cc/bundles/fa76eeebb40d9d9c38fc.js" >

AttributeError: 'NoneType' object has no attribute 'find_all'

In [187]:
res

0

In [119]:
region_name = {
    'Гатчина':  'gatchina',
    'Петрозаводск' : 'petrozavodsk',
    'Омск' : 'omsk',
}

In [6]:
house_type = {
    'Кирпичный': 'kirpichnyy_dom',
    'Панельный': 'panelnyy_dom',
    'Блочный': 'blochnyy_dom',
    'Монолитный': 'monolitnyy_dom',
    'Деревянный': 'derevyannyy_dom'
}

In [121]:
street = 'солнечный пер'

In [122]:
# Идёт учёт местоположения (до улицы), кол-ва комнат
base_url + region_name['Гатчина'] + '/kvartiry/prodam/' + rooms[1] + '/' + house_type['Панельный'] + '?q=' + '+'.join(street.split()) + '&s=104'

'https://www.avito.ru/gatchina/kvartiry/prodam/1-komnatnye/panelnyy_dom?q=солнечный+пер&s=104'

In [157]:
html_doc =  requests.get('https://www.avito.ru/kirovskaya_oblast_kirov/kvartiry/2-k._kvartira_429m_45et._2233086716')

In [172]:
str(html_doc) == '<Response [200]>'

True

In [159]:
# Первая страница поиска Авито
soup = BeautifulSoup(html_doc.text, "html.parser")

In [165]:
int(soup.find('span', class_ = 'js-item-price')['content'])

1780000

In [154]:
# Нахожу все подходящие (не рекомендации) квартиры на этой странице
flats = soup.find('div', class_ = 'items-items-kAJAg')  # получаю список квартир вместе с рекламой и рекомендациями
flats_act = flats.find_all(attrs={"data-marker": "item"})  # баблы страничек как массив

In [155]:
# Получаю сайт с каждой квартирой отдельно
flats_sites = []
for flat in flats_act:
    flats_sites.append('https://www.avito.ru' + flat.find(attrs={"class":"iva-item-sliderLink-bJ9Pv"})['href'])

In [156]:
flats_act

[]

In [128]:
flat_one_info = {
                'floor':None,
                'all_floors':None,
                'rooms':None,
                'all_square':None,
                'live_square':None,
                'kitch_square':None,
                'tall':None,
                'repair':None,
                'bathroom':None,
                'balcony':None,
                'window':None,
                'dop_info':None,
                'house_type':None,
                'year_built':None,
                'how_pas_lift':None,
                'how_grz_lift':None,
                'in_house':None,
                'backyard':None,
                'park':None
                }
df = pd.DataFrame(flat_one_info, index = [0])

In [129]:
for flat in flats_sites:
    flat_i = BeautifulSoup(requests.get(flat).text, "html.parser")
    
    flat_one_info = {
                    'floor':None,
                    'all_floors':None,
                    'rooms':None,
                    'all_square':None,
                    'live_square':None,
                    'kitch_square':None,
                    'tall':None,
                    'repair':None,
                    'bathroom':None,
                    'balcony':None,
                    'window':None,
                    'dop_info':None,
                    'house_type':None,
                    'year_built':None,
                    'how_pas_lift':None,
                    'how_grz_lift':None,
                    'in_house':None,
                    'backyard':None,
                    'park':None
                    }

    for text in flat_i.find_all('li', class_ = 'item-params-list-item'):
        label = text.text[:text.text.find(':')].split()[0]
        value = text.text[text.text.find(':') + 1:].split()
        if label == 'Этаж':
            flat_one_info['floor'] = value[0]
            flat_one_info['all_floors'] = value[2]
        elif label == 'Количество':
            flat_one_info['rooms'] = value[0]
        elif label == 'Общая':
            flat_one_info['all_square'] = value[0]
        elif label == 'Площадь':
            flat_one_info['live_square'] = value[0]
        elif label == 'Жилая':
            flat_one_info['kitchen_square'] = value[0]
        elif label == 'Высота':
            flat_one_info['tall'] = value[0]   
        elif label == 'Ремонт':
            flat_one_info['repair'] = value[0]
        elif label == 'Санузел':
            flat_one_info['bathroom'] = value[0]
        elif label == 'Балкон':
            flat_one_info['balcony'] = value[0]
        elif label == 'Вид':
            flat_one_info['window'] = ' '.join(value)
        elif label == 'Дополнительно':
            flat_one_info['dop_info'] = ' '.join(value)
        elif label == 'Тип':
            flat_one_info['house_type'] = value[0]
        elif label == 'Год':
            flat_one_info['year_built'] = value[0]
        elif label == 'Пассажирский':
            flat_one_info['how_pas_lift'] = value[0]
        elif label == 'Грузовой':
            flat_one_info['how_grz_lift'] = value[0]
        elif label == 'В':
            flat_one_info['in_house'] = ' '.join(value)
        elif label == 'Двор':
            flat_one_info['backyard'] = ' '.join(value)
        elif label == 'Парковка':
            flat_one_info['park'] = ' '.join(value) 
        
    df = df.append(flat_one_info, ignore_index=True)

In [130]:
df.drop(index = 0,inplace=True)

In [105]:
flat_one_info = {
                'floor':None,
                'all_floors':None,
                'rooms':None,
                'all_square':None,
                'live_square':None,
                'kitch_square':None,
                'tall':None,
                'repair':None,
                'bathroom':None,
                'balcony':None,
                'window':None,
                'dop_info':None,
                'house_type':None,
                'year_built':None,
                'how_pas_lift':None,
                'how_grz_lift':None,
                'in_house':None,
                'backyard':None,
                'park':None
                }

for text in flat_one.find_all('li', class_ = 'item-params-list-item'):
    label = text.text[:text.text.find(':')].split()[0]
    value = text.text[text.text.find(':') + 1:].split()
    if label == 'Этаж':
        flat_one_info['floor'] = value[0]
        flat_one_info['all_floors'] = value[2]
    elif label == 'Количество':
        flat_one_info['rooms'] = value[0]
    elif label == 'Общая':
        flat_one_info['all_square'] = value[0]
    elif label == 'Площадь':
        flat_one_info['live_square'] = value[0]
    elif label == 'Жилая':
        flat_one_info['kitchen_square'] = value[0]
    elif label == 'Высота':
        flat_one_info['tall'] = value[0]   
    elif label == 'Ремонт':
        flat_one_info['repair'] = value[0]
    elif label == 'Санузел':
        flat_one_info['bathroom'] = value[0]
    elif label == 'Балкон':
        flat_one_info['balcony'] = value[0]
    elif label == 'Вид':
        flat_one_info['window'] = ' '.join(value)
    elif label == 'Дополнительно':
        flat_one_info['dop_info'] = ' '.join(value)
    elif label == 'Тип':
        flat_one_info['house_type'] = value[0]
    elif label == 'Год':
        flat_one_info['year_built'] = value[0]
    elif label == 'Пассажирский':
        flat_one_info['how_pas_lift'] = value[0]
    elif label == 'Грузовой':
        flat_one_info['how_grz_lift'] = value[0]
    elif label == 'В':
        flat_one_info['in_house'] = ' '.join(value)
    elif label == 'Двор':
        flat_one_info['backyard'] = ' '.join(value)
    elif label == 'Парковка':
        flat_one_info['park'] = ' '.join(value)
#    print(label)
#    print(value)

In [102]:
flat_one.find_all('li', class_ = 'item-params-list-item')

[<li class="item-params-list-item"> <span class="item-params-label">Этаж: </span>1 из 5 </li>,
 <li class="item-params-list-item"> <span class="item-params-label">Количество комнат: </span>1 </li>,
 <li class="item-params-list-item"> <span class="item-params-label">Общая площадь: </span>42.1 м² </li>,
 <li class="item-params-list-item"> <span class="item-params-label">Площадь кухни: </span>10.1 м² </li>,
 <li class="item-params-list-item"> <span class="item-params-label">Жилая площадь: </span>17.3 м² </li>,
 <li class="item-params-list-item"> <span class="item-params-label">Высота потолков: </span>2.7 м </li>,
 <li class="item-params-list-item"> <span class="item-params-label">Ремонт: </span>косметический </li>,
 <li class="item-params-list-item"> <span class="item-params-label">Санузел: </span>раздельный </li>,
 <li class="item-params-list-item"> <span class="item-params-label">Балкон или лоджия: </span>лоджия </li>,
 <li class="item-params-list-item"> <span class="item-params-label">

In [101]:
flat_one_info = {}
for text in flat_one.find_all('li', class_ = 'item-params-list-item'):
#    print(text.text)
    print(text.text)
    label = text.text[:text.text.find(':')].split()[0]
    value = text.text[text.text.find(':') + 1:].split()
    
#    print(label)
    if label == 'Этаж':
        flat_one_info[label] = value[0]
        flat_one_info['all_floors'] = value[2]
    elif label in ('Вид', 'Дополнительно', 'Двор', 'Парковка', 'В'):
        flat_one_info[label] = ' '.join(value)
    elif label == 'В':
        flat_one_info['in_house'] = value[0]
    else:
        flat_one_info[label] = value[0]

 Этаж: 1 из 5 
 Количество комнат: 1 
 Общая площадь: 42.1 м² 
 Площадь кухни: 10.1 м² 
 Жилая площадь: 17.3 м² 
 Высота потолков: 2.7 м 
 Ремонт: косметический 
 Санузел: раздельный 
 Балкон или лоджия: лоджия 
 Вид из окон: на улицу, во двор 
 Дополнительно: мебель, бытовая техника, панорамные окна 
 Тип дома: кирпичный 
 Год постройки: 1990 
 Пассажирский лифт: 0 
 Грузовой лифт: 0 
 В доме: газоснабжение 
 Двор: детская площадка 
 Парковка: открытая во дворе 


In [85]:
df = pd.DataFrame(flat_one_info, index = [0])

In [86]:
df

,Этаж,all_floors,Количество,Общая,Площадь,Жилая,Высота,Ремонт,Санузел,Балкон,Вид,Дополнительно,Тип,Год,Пассажирский,Грузовой,В,Двор,Парковка
0,1,5,1,42.1,10.1,17.3,2.7,косметический,раздельный,лоджия,"на улицу, во двор","мебель, бытовая техника, панорамные окна",кирпичный,1990,0,0,газоснабжение,детская площадка,открытая во дворе


In [125]:
for flat in flats:
    flat.find()

<div class="items-ads-cA1GS" elementtiming="bx.gallery.first-item"><div class="styles-positionContext-_XkUo"><div class="styles-avitoAdsContexContainer-lPBSB avito-ads-container_context_1 styles-avitoAdsContainer_withPlaceholder-YkKlN"><div class="styles-avitoAdsTemplate-_HpwS" id="template_context_1"><div class="avito-ads-content"></div></div></div><div class="styles-skeleton-_RUnA"></div></div></div>
<div class="iva-item-root-Nj_hb photo-slider-slider-_PvpN iva-item-list-H_dpX iva-item-redesign-nV4C4 iva-item-responsive-gIKjW items-item-My3ih items-listItem-Gd1jN js-catalog-item-enum" data-item-id="2218320258" data-marker="item" id="i2218320258" itemscope="" itemtype="http://schema.org/Product"><meta content="Срочная Продажа (после 20 октября цена вырастет) В связи с переездом в другой город! Мечтаете жить в центре Омска, при этом в тихом, спокойном месте. Тогда вы нашли то, что искали. Предлагаю уютную квартиру в хорошем состоянии, на полу ламинат, в кухне на " itemprop="description

In [107]:
# Список найденных квартир на продажу
flats.find_all("a", "class" == "iva-item-sliderLink-bJ9Pv")

[<a class="iva-item-sliderLink-bJ9Pv" href="/omsk/kvartiry/1-k._kvartira_421m_15et._2218320258" itemprop="url" rel="noopener" target="_blank" title="Объявление «1-к. квартира, 42,1 м², 1/5 эт.» 14 фотографий"><div class="photo-slider-root-w2CZ2 photo-slider-redesign-oqzmv" data-marker="item-photo"><div class="photo-slider-photoSlider-IpiW8 photo-slider-aspect-ratio-4-3-Jhpvf"><ul class="photo-slider-list-xFf2c"><li class="photo-slider-list-item-_fUPr" data-marker="slider-image/image-https://99.img.avito.st/208x156/11573669299.jpg"><div class="photo-slider-item-o_UGQ photo-slider-keepImageRatio-NrG6s"><img alt="1-к. квартира, 42,1 м², 1/5 эт." class="photo-slider-image-_Dc4I" importance="high" itemprop="image" sizes="
                 (min-width: 1334px) 236px,
                 208px
             " src="https://99.img.avito.st/208x156/11573669299.jpg" srcset="https://99.img.avito.st/208x156/11573669299.jpg 208w,https://99.img.avito.st/image/1/br6Xs7a-wlf5EyJXx6YByHcQxFE3EsBdKcjDtyMQwFcj

In [56]:
# Полный список постраничных блоков квартир
res = soup.find_all(attrs={"data-marker": "item"})

In [54]:
soup.find(attrs={"data-marker": "item"})

<div class="iva-item-root-Nj_hb photo-slider-slider-_PvpN iva-item-list-H_dpX iva-item-redesign-nV4C4 iva-item-responsive-gIKjW items-item-My3ih items-listItem-Gd1jN js-catalog-item-enum" data-item-id="2234063088" data-marker="item" id="i2234063088" itemscope="" itemtype="http://schema.org/Product"><meta content='Продается отличная 3-х комнатная квартира в ЖК "Тарская крепость" расположенная на 7-м этаже в 10-ти этажном панельном доме. Квартира в хорошем состоянии, окна Пвх и дерево, лоджия застеклена и утеплена, комнаты изолированные, санузел раздельный, стены выл' itemprop="description"/><div class="iva-item-content-UnQQ4"><div class="iva-item-slider-GWoCM"><a class="iva-item-sliderLink-bJ9Pv" href="/omsk/kvartiry/3-k._kvartira_821m_710et._2234063088" itemprop="url" rel="noopener" target="_blank" title="Объявление «3-к. квартира, 82,1 м², 7/10 эт.» 18 фотографий"><div class="photo-slider-root-w2CZ2 photo-slider-redesign-oqzmv" data-marker="item-photo"><div class="photo-slider-photoSl

In [31]:
for tag in soup.find_all('div', data-marker = 'item'):
    print('huu')

SyntaxError: expression cannot contain assignment, perhaps you meant "=="? (<ipython-input-31-897911ba7107>, line 1)

In [11]:
soup.find('span', class_ = 'geo-address-QTv9k text-text-LurtD text-size-s-BxGpL').text

'ул. Варламова, 37'

In [ ]:
soup.find_all('div', class_ = 'iva-item-developmentNameStep-n46gZ')

In [ ]:
geo-address-QTv9k text-text-LurtD text-size-s-BxGpL